In [1]:
import pandas as pd
import clickhouse_connect

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.width = None
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 20)

In [2]:
# 10 x 5
states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia",
            "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland",
            "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey",
            "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina",
            "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

CLICKHOUSE_CLOUD_HOSTNAME = 'csi4142-clickhouse.mershab.xyz'
CLICKHOUSE_CLOUD_USER = 'csi'
CLICKHOUSE_CLOUD_PASSWORD = 'iceberg'

TABLE_NAME1 = 'median_household_income_dimension_table_test'
TABLE_NAME2 = 'cost_of_living_dimension_table_test'

In [3]:
# Yearly Median Household Income (USD) by State from 1984 - 2022

df_list_income = []
single_ticker_df = {}
for state in states:
    temp_df = pd.DataFrame(pd.read_csv(f"../data/income_data/{state}Income.csv")) #, parse_dates=['datetime']
    
    temp_df.columns.values[1] = "Median_Household_Income_USD"
    temp_df = temp_df.rename(columns={"DATE": "Year"})

    temp_df['Year'] = pd.DatetimeIndex(temp_df['Year']).year
    temp_df['State'] = state
    df_list_income.append(temp_df) 


df_list_income

[    Year  Median_Household_Income_USD    State
 0   1984                        43850  Alabama
 1   1985                        44910  Alabama
 2   1986                        46110  Alabama
 3   1987                        46010  Alabama
 4   1988                        44850  Alabama
 5   1989                        45840  Alabama
 6   1990                        47970  Alabama
 7   1991                        48250  Alabama
 8   1992                        49880  Alabama
 9   1993                        47310  Alabama
 10  1994                        50200  Alabama
 11  1995                        46860  Alabama
 12  1996                        53230  Alabama
 13  1997                        54930  Alabama
 14  1998                        61530  Alabama
 15  1999                        60260  Alabama
 16  2000                        56920  Alabama
 17  2001                        55250  Alabama
 18  2002                        58360  Alabama
 19  2003                        56640  

In [4]:
# Data for Cost of Living Index runs from 1960-2003, 2016-2017, 2019-2023
# Data for Cost of Living Index unavailable for certain states prior to 2016 (Hawaii, Alaska)

# "location","indicator","Unit","Date","Value"

# Data for Indicators ("Housing", "Groceries", "Utilities", "Transportation", "Miscellaneous", "Health Care") from 2017, 2019-2023

df_list_col = []
single_ticker_df = {}

temp_df = pd.DataFrame(pd.read_csv(f"../data/cost_of_living_data/cost_of_living_data.csv"))
temp_df = temp_df.drop(columns=['Unit'])
temp_df = temp_df.rename(columns={"Date": "Year", "Value": "Cost_of_Living_Index", "location": "State", "indicator": "Indicator"})

temp_df['Year'] = pd.DatetimeIndex(temp_df['Year']).year
df_list_col.append(temp_df) 

df_list_col

[        State             Indicator  Year  Cost_of_Living_Index
 0      Alaska               Housing  1970             140.30000
 1      Alaska               Housing  1970             134.60000
 2      Alaska               Housing  1970             130.80000
 3      Alaska               Housing  1970             126.90000
 4      Alaska               Housing  1970             121.30000
 5      Alaska               Housing  1970             122.50000
 6      Alaska               Grocery  1970             135.70000
 7      Alaska               Grocery  1970             134.00000
 8      Alaska               Grocery  1970             128.70000
 9      Alaska               Grocery  1970             134.50000
 ...       ...                   ...   ...                   ...
 4295  Wyoming  Cost of Living Index  1970              91.61721
 4296  Wyoming  Cost of Living Index  1970              93.86558
 4297  Wyoming  Cost of Living Index  1970              96.63952
 4298  Wyoming  Cost of L

In [5]:
df_income = pd.concat(df_list_income)

df_income

,Year,Median_Household_Income_USD,State
0,1984,43850,Alabama
1,1985,44910,Alabama
2,1986,46110,Alabama
3,1987,46010,Alabama
4,1988,44850,Alabama
5,1989,45840,Alabama
6,1990,47970,Alabama
7,1991,48250,Alabama
8,1992,49880,Alabama
9,1993,47310,Alabama


In [6]:
df_col = pd.concat(df_list_col)

df_col

,State,Indicator,Year,Cost_of_Living_Index
0,Alaska,Housing,1970,140.30000
1,Alaska,Housing,1970,134.60000
2,Alaska,Housing,1970,130.80000
3,Alaska,Housing,1970,126.90000
4,Alaska,Housing,1970,121.30000
5,Alaska,Housing,1970,122.50000
6,Alaska,Grocery,1970,135.70000
7,Alaska,Grocery,1970,134.00000
8,Alaska,Grocery,1970,128.70000
9,Alaska,Grocery,1970,134.50000


In [7]:
print(df_col.dtypes)
print(df_income.dtypes)

State                    object
Indicator                object
Year                      int32
Cost_of_Living_Index    float64
dtype: object
Year                            int32
Median_Household_Income_USD     int64
State                          object
dtype: object


In [8]:
client = clickhouse_connect.get_client(host=CLICKHOUSE_CLOUD_HOSTNAME, port=443, user=CLICKHOUSE_CLOUD_USER, password=CLICKHOUSE_CLOUD_PASSWORD,database='default')

print(client.ping())
print("connected to " + CLICKHOUSE_CLOUD_HOSTNAME + "\n")

True
connected to csi4142-clickhouse.mershab.xyz



In [19]:
# Check if table exists and create if it doesn't
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {TABLE_NAME2} (
        State         String,
        Indicator     String,
        Year          UInt32,
        Cost_of_Living_Index  Float64
) ENGINE = MergeTree()
ORDER BY Year
"""

#client.command(f'DROP TABLE IF EXISTS {TABLE_NAME2}')
client.command(create_table_query)

In [28]:
create_table_query2 = f"""
CREATE TABLE IF NOT EXISTS {TABLE_NAME1} (
        Year                              UInt32,
        Median_Household_Income_USD       Float64,
        State                             String
) ENGINE = MergeTree()
ORDER BY Year
"""
#client.command(f'DROP TABLE IF EXISTS {TABLE_NAME1}')
client.command(create_table_query2)

In [20]:
client.insert_df(TABLE_NAME2, df_col)

In [30]:
client.insert_df(TABLE_NAME1, df_income)